In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Parameters
VIX_THR = 15
START = "2010-01-01"
END = "2025-12-31"

# -----------------------------
# Download data
# -----------------------------
vix = yf.download("^VIX", start=START, end=END, progress=False)
upro = yf.download("UPRO", start=START, end=END, progress=False)

print("VIX rows:", len(vix))
print("UPRO rows:", len(upro))

# Check for errors
if vix.empty:
    raise ValueError("VIX data download failed — dataset is EMPTY.")
if upro.empty:
    raise ValueError("UPRO data download failed — dataset is EMPTY.")

# -----------------------------
# Merge using index to avoid scalar error
# -----------------------------
df = pd.merge(
    vix[["Close"]].rename(columns={"Close": "VIX"}),
    upro[["Close"]].rename(columns={"Close": "UPRO"}),
    left_index=True,
    right_index=True,
    how="inner"
)

df.dropna(inplace=True)

print(df.head())
print(df.tail())


In [ ]:
print("VIX index:", df["VIX"].index[:10])
print("MA5 index:", df["VIX_MA5"].index[:10])

print("VIX length:", len(df["VIX"]))
print("MA5 length:", len(df["VIX_MA5"]))


In [5]:
import yfinance as yf
import pandas as pd
import numpy as np

VIX_THR = 15
START = "2010-01-01"
END = "2025-12-31"

# Download data
vix = yf.download("^VIX", start=START, end=END, progress=False)
upro = yf.download("UPRO", start=START, end=END, progress=False)

# Flatten columns if multi-index exists
if isinstance(vix.columns, pd.MultiIndex):
    vix.columns = [f"{i}_{j}" for i,j in vix.columns]  # e.g., 'Close_^VIX'
if isinstance(upro.columns, pd.MultiIndex):
    upro.columns = [f"{i}_{j}" for i,j in upro.columns]  # e.g., 'Close_UPRO'

# Rename to simple columns
vix.rename(columns={col: "VIX" for col in vix.columns if "Close" in col}, inplace=True)
upro.rename(columns={col: "UPRO" for col in upro.columns if "Close" in col}, inplace=True)

# Merge on index
df = pd.merge(vix[["VIX"]], upro[["UPRO"]], left_index=True, right_index=True, how="inner")

# Indicators
df["VIX_MA5"] = df["VIX"].rolling(5).mean()

# Drop NaN from MA5
df = df.dropna(subset=["VIX_MA5"])

# Strategy rules
enter = (df["VIX"] < VIX_THR) & (df["VIX"] < df["VIX_MA5"])
exit_  = (df["VIX"] > VIX_THR) & (df["VIX"] > df["VIX_MA5"])

# Position logic
position = []
pos = 0
for i in range(len(df)):
    if pos == 0:
        if enter.iloc[i]:
            pos = 1
    else:
        if exit_.iloc[i]:
            pos = 0
    position.append(pos)

df["position"] = position

df.head()


/tmp/ipykernel_16959/1535400948.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  vix = yf.download("^VIX", start=START, end=END, progress=False)
/tmp/ipykernel_16959/1535400948.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  upro = yf.download("UPRO", start=START, end=END, progress=False)


,VIX,UPRO,VIX_MA5,position
Date,,,,
2010-01-08,18.129999,2.147543,19.148,0
2010-01-11,17.549999,2.155158,18.650,0
2010-01-12,18.250000,2.096611,18.430,0
2010-01-13,17.850000,2.150038,18.168,0
2010-01-14,17.629999,2.166053,17.882,0


In [6]:
# -----------------------------
# Daily returns
# -----------------------------
df["UPRO_ret"] = df["UPRO"].pct_change().fillna(0)

# Strategy return = UPRO return * position
df["strategy_ret"] = df["UPRO_ret"] * df["position"]

# NAV calculation
df["NAV_strategy"] = (1 + df["strategy_ret"]).cumprod()
df["NAV_buyhold"] = (1 + df["UPRO_ret"]).cumprod()

In [7]:
# -----------------------------
# Performance statistics
# -----------------------------
def annualized_return(nav):
    total_ret = nav.iloc[-1] / nav.iloc[0]
    N = len(nav)
    return total_ret ** (252 / N) - 1

def annualized_vol(returns):
    return returns.std() * np.sqrt(252)

def sharpe(ret, vol):
    return ret / vol if vol != 0 else np.nan

def max_drawdown(nav):
    roll_max = nav.cummax()
    dd = nav / roll_max - 1
    mdd = dd.min()
    end = dd.idxmin()
    start = nav.loc[:end].idxmax()
    return mdd, start, end

In [8]:
# Metrics
ann_ret = annualized_return(df["NAV_strategy"])
ann_vol = annualized_vol(df["strategy_ret"])
sharpe_ratio = sharpe(ann_ret, ann_vol)
mdd, mdd_start, mdd_end = max_drawdown(df["NAV_strategy"])

ann_ret_bh = annualized_return(df["NAV_buyhold"])
ann_vol_bh = annualized_vol(df["UPRO_ret"])
sharpe_bh = sharpe(ann_ret_bh, ann_vol_bh)
mdd_bh, mdd_bh_start, mdd_bh_end = max_drawdown(df["NAV_buyhold"])

In [9]:
# Output results
# -----------------------------
print("=== Strategy (VIX rule) ===")
print(f"Final NAV: {df['NAV_strategy'].iloc[-1]:.4f}")
print(f"Annualized return: {ann_ret*100:.2f}%")
print(f"Annualized vol: {ann_vol*100:.2f}%")
print(f"Sharpe (rf=0.0): {sharpe_ratio:.2f}")
print(f"Max Drawdown: {mdd*100:.2f}%  (from {mdd_start.date()} to {mdd_end.date()})")
print()

print("=== Buy & Hold UPRO ===")
print(f"Final NAV: {df['NAV_buyhold'].iloc[-1]:.4f}")
print(f"Annualized return: {ann_ret_bh*100:.2f}%")
print(f"Annualized vol: {ann_vol_bh*100:.2f}%")
print(f"Sharpe (rf=0.0): {sharpe_bh:.2f}")
print(f"Max Drawdown: {mdd_bh*100:.2f}%  (from {mdd_bh_start.date()} to {mdd_bh_end.date()})")
print()

print("Trades executed:", (df["position"].diff().abs().sum()))
print("Total days in market:", df["position"].sum())
print()

print("Recent data:")
print(df.tail(10))


=== Strategy (VIX rule) ===
Final NAV: 621.1718
Annualized return: 50.05%
Annualized vol: 14.83%
Sharpe (rf=0.0): 3.37
Max Drawdown: -8.79%  (from 2013-08-02 to 2013-08-16)

=== Buy & Hold UPRO ===
Final NAV: 49.0700
Annualized return: 27.84%
Annualized vol: 51.67%
Sharpe (rf=0.0): 0.54
Max Drawdown: -76.82%  (from 2020-02-19 to 2020-03-23)

Trades executed: 202.0
Total days in market: 1425

Recent data:
                  VIX        UPRO  VIX_MA5  position  UPRO_ret  strategy_ret  \
Date                                                                           
2025-11-10  17.600000  117.120003   18.638         0  0.045248           0.0   
2025-11-11  17.280001  117.930000   18.294         0  0.006916           0.0   
2025-11-12  17.510000  118.129997   18.194         0  0.001696           0.0   
2025-11-13  20.000000  112.250000   18.294         0 -0.049776          -0.0   
2025-11-14  19.830000  112.080002   18.444         0 -0.001514          -0.0   
2025-11-17  22.379999  108.91000